In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import torch
from tqdm.auto import tqdm
from sklearn.linear_model import RidgeCV
from utils import *

device='cpu'

def logit(x, eps=.001):
    c = 0 #np.min(x)
    x2 = (np.array(x)-c)/(1-c)
    return np.log((x2+eps)/(1-x2+eps))

data_path = '/llmthonskdir/felipe/download_openllmlb/'

mmlu_subs = ['hendrycksTest-abstract_algebra', 'hendrycksTest-anatomy', 'hendrycksTest-astronomy', 'hendrycksTest-business_ethics',
             'hendrycksTest-clinical_knowledge', 'hendrycksTest-college_biology', 'hendrycksTest-college_chemistry',
             'hendrycksTest-college_computer_science', 'hendrycksTest-college_mathematics', 'hendrycksTest-college_medicine', 'hendrycksTest-college_physics', 'hendrycksTest-computer_security', 'hendrycksTest-conceptual_physics', 'hendrycksTest-econometrics', 'hendrycksTest-electrical_engineering', 'hendrycksTest-elementary_mathematics', 'hendrycksTest-formal_logic', 'hendrycksTest-global_facts', 'hendrycksTest-high_school_biology', 'hendrycksTest-high_school_chemistry', 'hendrycksTest-high_school_computer_science', 'hendrycksTest-high_school_european_history', 'hendrycksTest-high_school_geography', 'hendrycksTest-high_school_government_and_politics', 'hendrycksTest-high_school_macroeconomics', 'hendrycksTest-high_school_mathematics', 'hendrycksTest-high_school_microeconomics', 'hendrycksTest-high_school_physics', 'hendrycksTest-high_school_psychology', 'hendrycksTest-high_school_statistics', 'hendrycksTest-high_school_us_history', 'hendrycksTest-high_school_world_history', 'hendrycksTest-human_aging', 'hendrycksTest-human_sexuality', 'hendrycksTest-international_law', 'hendrycksTest-jurisprudence', 'hendrycksTest-logical_fallacies', 'hendrycksTest-machine_learning', 'hendrycksTest-management', 'hendrycksTest-marketing', 'hendrycksTest-medical_genetics', 'hendrycksTest-miscellaneous', 'hendrycksTest-moral_disputes', 'hendrycksTest-moral_scenarios', 'hendrycksTest-nutrition', 'hendrycksTest-philosophy', 'hendrycksTest-prehistory', 'hendrycksTest-professional_accounting', 'hendrycksTest-professional_law', 'hendrycksTest-professional_medicine', 'hendrycksTest-professional_psychology', 'hendrycksTest-public_relations', 'hendrycksTest-security_studies', 'hendrycksTest-sociology', 'hendrycksTest-us_foreign_policy', 'hendrycksTest-virology', 'hendrycksTest-world_religions']

In [2]:
data = pd.read_csv('data/base_llm_benchmark_eval.csv')
models_scaling = list(data.Model)
models_scaling = [filter(m) for m in models_scaling]

with open(data_path+'scaling_laws/old_leaderboard_processed_20240630.pickle', 'rb') as handle:
    lb_data = pickle.load(handle)
models_lb = lb_data['hendrycksTest-abstract_algebra']['models']
models_lb = [filter(m) for m in models_lb]

accs_mmlu = [lb_data[s]['correctness'].mean(1) for s in mmlu_subs]
accs_mmlu = np.vstack(accs_mmlu).T

In [3]:
with open(data_path+'/old_leaderboard_processed_20240630.pickle', 'rb') as handle:
    full_lb_data = pickle.load(handle)
asymptot = {}
for s in full_lb_data.keys():
    asymptot[s] = np.percentile(full_lb_data[s]['correctness'].mean(-1), 1)

In [4]:
for j,s in enumerate(mmlu_subs):
    posics = []
    for m in models_scaling:
        if m in models_lb:
            posics.append(np.array(np.array(models_lb)==m).argmax())
        else:
            posics.append(-1)
    
    y = []
    for posic in posics:
        if posic==-1:
            y.append(np.nan)
        else:
            y.append(accs_mmlu[posic,j])
            
    data[s] = y

In [5]:
fam_encoder = LabelEncoder()
fam_encoder.fit(data['Model Family'])
data['T'] = data['Pretraining Data Size (T)']
data['S'] = data['Model Size (B)']
data['F'] = data['FLOPs (1E21)']
data['family'] = data['Model Family']
data = data.sort_values(by=['family','S']).reset_index(drop=True)
data['logT'] = np.log(data['T'])
data['logS'] = np.log(data['S'])
data['logF'] = np.log(data['F'])
data['logS*logT'] = data['logS']*data['logT']
data = data[['family','logT','logS','logF','logS*logT','ARC-C','HellaSwag','Winograd','TruthfulQA','GSM8K']+mmlu_subs] #,'XWinograd','HumanEval'

In [6]:
def sigmoid(z, use_torch=True):
    if use_torch:
        return torch.nn.Sigmoid()(z)
    else:
        return 1/(1+np.exp(-z))

In [7]:
d = 10

epochs=100
tol=1e-8

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


In [9]:
data = data.dropna(how='any')
unique_families, counts_families = np.unique(data.family, return_counts=True)
avail_families = unique_families[counts_families>=2]

In [12]:
preds = []
preds_baseline = []
preds_baseline2 = []
preds_rf = []
ys = []

for test_family in tqdm(avail_families):
    
    data_train = data.loc[data.family != test_family]
    data_test = data.loc[data.family == test_family]
    data_train = pd.concat((data_test.iloc[:1],data_train), axis=0).reset_index(drop=True)
    data_test = data_test.iloc[1:].reset_index(drop=True)
    
    ###
    Y_train = torch.tensor(logit(np.array(data_train.loc[:,['ARC-C','HellaSwag','Winograd','TruthfulQA','GSM8K']+mmlu_subs])))
    X_train = np.array(data_train.loc[:,['logT','logS','logS*logT']])
    X_train = torch.tensor(np.hstack((np.ones(X_train.shape[0])[:,None], X_train))).double()
    Y_test = torch.tensor(logit(np.array(data_test.loc[:,['ARC-C','HellaSwag','Winograd','TruthfulQA','GSM8K']+mmlu_subs])))
    X_test = np.array(data_test.loc[:,['logT','logS','logS*logT']])
    X_test = torch.tensor(np.hstack((np.ones(X_test.shape[0])[:,None], X_test))).double()
    D_train = torch.tensor(np.array(pd.get_dummies(np.array(data_train.family)))).double()
    D_test = torch.tensor(np.vstack([D_train[0,:].numpy() for _ in range(Y_test.shape[0])])).double()

    X2_train = torch.hstack((torch.tensor(np.array(data_train.loc[:,['logT','logS','logS*logT']])), D_train)) #torch.tensor(np.array(data_train.loc[:,['logF']]))
    X2_test = torch.hstack((torch.tensor(np.array(data_test.loc[:,['logT','logS','logS*logT']])), D_test)) #torch.tensor(np.array(data_train.loc[:,['logF']]))

    ###
    thetas = torch.nn.Parameter(torch.normal(0, .5, size=(D_train.shape[1],d), dtype=torch.float64, device=device))
    beta = torch.nn.Parameter(torch.normal(0, .5, size=(X_train.shape[1]+d,Y_train.shape[1]), dtype=torch.float64, device=device))
    beta_baseline = torch.nn.Parameter(torch.normal(0, .5, size=(X_train.shape[1],Y_train.shape[1]), dtype=torch.float64, device=device))
    beta_baseline2 = torch.nn.Parameter(torch.normal(0, .5, size=(X2_train.shape[1],Y_train.shape[1]), dtype=torch.float64, device=device))
    
    ###
    optimizer = torch.optim.LBFGS([beta,thetas], lr=.1, line_search_fn='strong_wolfe')
    losses = []
    for epoch in range(epochs):
        def closure():
            optimizer.zero_grad()
            loss = (((X_train@beta[d:] + (D_train@thetas)@beta[:d])-Y_train)**2).mean()
            loss.backward()
            return loss
        loss = optimizer.step(closure)
        abs_loss = (((X_train@beta[d:] + (D_train@thetas)@beta[:d])-Y_train).abs()).mean()
        losses.append(abs_loss.item())
    
        if epoch>=1:
            if losses[-2]-losses[-1]<=tol:
                break

    ###
    optimizer = torch.optim.LBFGS([beta_baseline], lr=.1, line_search_fn='strong_wolfe')
    losses = []
    for epoch in range(epochs):
        def closure():
            optimizer.zero_grad()
            loss = (((X_train@beta_baseline)-Y_train)**2).mean()
            loss.backward()
            return loss
        loss = optimizer.step(closure)
        abs_loss = (((X_train@beta_baseline)-Y_train).abs()).mean()
        losses.append(abs_loss.item())
    
        if epoch>=1:
            if losses[-2]-losses[-1]<=tol:
                break

    ###
    optimizer = torch.optim.LBFGS([beta_baseline2], lr=.1, line_search_fn='strong_wolfe')
    losses = []
    for epoch in range(epochs):
        def closure():
            optimizer.zero_grad()
            loss = (((X2_train@beta_baseline2)-Y_train)**2).mean()
            loss.backward()
            return loss
        loss = optimizer.step(closure)
        abs_loss = (((X2_train@beta_baseline2)-Y_train).abs()).mean()
        losses.append(abs_loss.item())
    
        if epoch>=1:
            if losses[-2]-losses[-1]<=tol:
                break

    preds.append(sigmoid(((X_test@beta[d:] + (D_test@thetas)@beta[:d]))).detach().numpy())
    preds_baseline.append(sigmoid(((X_test@beta_baseline))).detach().numpy())
    preds_baseline2.append(sigmoid(((X2_test@beta_baseline2))).detach().numpy())

    ###
    cols = [[0],[1],[2],[3],[4],list(range(Y_train.shape[1]))[5:]]
    pred_rf = np.zeros((Y_test.shape[0],Y_train.shape[1]))
    for col in cols:
        regr = RidgeCV(alphas=np.linspace(1e-5,10,20)) #RandomForestRegressor(n_estimators=1000, max_features='sqrt', random_state=0)
        regr.fit(Y_train[:,:4], Y_train[:,col].squeeze()) #np.delete(sigmoid(Y_train), col, axis=1), 
        pred_rf[:,col] = sigmoid(regr.predict(logit(preds_baseline2[-1])[:,:4]).reshape(-1,len(col)), use_torch=False) #np.delete(logit(preds_baseline2[-1]), col, axis=1)
        #pred_rf[:,[i for i in range(Y_train.shape[1]) if i not in col]] = preds_baseline2[-1][:,[i for i in range(Y_train.shape[1]) if i not in col]]
    preds_rf.append(pred_rf)
        
    ys.append(sigmoid(Y_test[:,:]).numpy())

  0%|          | 0/22 [00:00<?, ?it/s]

In [13]:
np.mean(np.abs(np.vstack(preds)[:,:5]-np.vstack(ys)[:,:5]), axis=0)

array([0.03484117, 0.04405401, 0.02929322, 0.04246133, 0.05998127])

In [14]:
np.mean(np.abs(np.vstack(preds_baseline)[:,:5]-np.vstack(ys)[:,:5]), axis=0)

array([0.05159197, 0.06620617, 0.04097824, 0.03716612, 0.0777711 ])

In [15]:
np.mean(np.abs(np.vstack(preds_baseline2)[:,:5]-np.vstack(ys)[:,:5]), axis=0)

array([0.02932462, 0.046043  , 0.02988084, 0.04813632, 0.06037273])

In [16]:
np.mean(np.abs(np.vstack(preds_rf)[:,:5]-np.vstack(ys)[:,:5]), axis=0)

array([0.02936353, 0.04601182, 0.02983732, 0.04809264, 0.09816683])

In [17]:
np.median(np.abs(np.vstack(preds)[:,:5]-np.vstack(ys)[:,:5]), axis=0)

array([0.02349686, 0.029052  , 0.02275559, 0.03488756, 0.01179707])

In [18]:
np.median(np.abs(np.vstack(preds_baseline)[:,:5]-np.vstack(ys)[:,:5]), axis=0)

array([0.03058035, 0.04576952, 0.02934638, 0.02100441, 0.02115813])

In [19]:
np.median(np.abs(np.vstack(preds_baseline2)[:,:5]-np.vstack(ys)[:,:5]), axis=0)

array([0.01784383, 0.03382038, 0.02087774, 0.04381062, 0.01115699])

In [20]:
np.median(np.abs(np.vstack(preds_rf)[:,:5]-np.vstack(ys)[:,:5]), axis=0)

array([0.01797291, 0.03410773, 0.02103038, 0.04392663, 0.02196258])

In [21]:
np.mean(np.abs(np.vstack(preds)[:,5:].mean(1)-np.vstack(ys)[:,5:].mean(1)))

0.05534097995038255

In [22]:
np.mean(np.abs(np.vstack(preds_baseline)[:,5:].mean(1)-np.vstack(ys)[:,5:].mean(1)))

0.07484974350315568

In [23]:
np.mean(np.abs(np.vstack(preds_baseline2)[:,5:].mean(1)-np.vstack(ys)[:,5:].mean(1)))

0.05556360186100937

In [24]:
np.mean(np.abs(np.vstack(preds_rf)[:,5:].mean(1)-np.vstack(ys)[:,5:].mean(1)), axis=0)

0.07491827144189878

In [21]:
np.median(np.abs(np.vstack(preds)[:,5:].mean(1)-np.vstack(ys)[:,5:].mean(1)))

0.04061560915989798

In [22]:
np.median(np.abs(np.vstack(preds_baseline)[:,5:].mean(1)-np.vstack(ys)[:,5:].mean(1)))

0.07036375818993645

In [23]:
np.median(np.abs(np.vstack(preds_baseline2)[:,5:].mean(1)-np.vstack(ys)[:,5:].mean(1)))

0.04036635411883588